# Monte Carlo 2D Ising Model

Authors: Chris King, James Grant

This tutorial aims to help solidify your understanding of the theory underlying the Monte Carlo simulation technique by applying it to model the magnetic properties of a 2D material.  

In [5]:
# import everything that we will need in this tutorial now
import numpy
import matplotlib.pyplot as plt
from inputs.Tut_2.sources.ising import IsingModel
from inputs.Tut_2.sources.isingdata import IsingModelData

## Introduction to Monte Carlo Methods:

Monte Carlo (MC) is the name given to the simulation technique that attempts to solve a problem by randomly sampling out of all of its possible outcomes ('configurational space')and obtaining a result based on numerical analysis of the sampling.  MC is a stochastic method, which means that the final state of the system cannot be predicted precisely based on the initial state and parameters, but through numerical analysis, reproducible results can be obtained.  This contrasts with other techniques like molecular dynamics, which are deterministic, where if you know the initial state and the inputs for the calculations, you can predict what the configuration of the system will be at any and all times thereafter.  This distinction allows MC to be used in a variety of applications across the scientific community where deterministic techniques are ineffective or impossible to use, such as phase co-existence and criticality, adsorption, and development of solid-state defects [1].

Results from MC simulations are generally accurate and reliable, assuming that the technique has representatively sampled the distribution of possible configurations in the system ('configurational space').  In other words, if our sampling method returns the probability distribution we expect, then we know that are sampling method is reliable.  In thermodynamic systems, the probability distribution of available states is given by the Boltzmann distribution:

$$W(\mathbf{r}) = \exp {\Bigl(\frac{E}{kT}\Bigr)}$$

where $W(\mathbf{r})$ is the probability of being in a state of energy, also known as the statistical weight, *E*, at temperature, *T*, and *k* is the Boltzmann constant.  The ratio of Boltzmann distributions at two different energies, $E_2$ and $E_1$, is known as the Boltzmann factor:
  
$$\frac{W(\mathbf{r}_1)}{W(\mathbf{r}_2)} = \exp {\Bigl(\frac{E_2 -E_1}{kT}\Bigr)}$$

So if our sampling method yields the Boltzmann distribution, we know that our simulation accurately reflects real systems.  There are many possible ways one can sample the configurational space of a simulated system, the intuitive case is simple random sampling in that we move randomly from one configuration to another.  However, this process is only reliable in systems with a constant probability distribution of states as it does not take into account the respective weighting of a given configuration.  For example, it can under-represent a small number of configurations who contribute significantly to the overall state of the system.

The concept of statistical weight is crucial in thermodynamics and describes how likely a particular configuration is of being observed out of a hypothetically *large* number of replicas of that system.  For instance, consider the possible configurations of the gas molecules in this room, clearly, this system would have a high probability of being in a configuration where the gas molecules are evenly (on average) distributed throughout the volume of the room and so this configuration has a high weighting.  Yet, there is a configuration where every gas molecule sits in one corner of the room, this configuration is highly unlikely to be seen and so its weighting is very low.  The weight of a particular configuration is given by:

$$W(\mathbf{r}) = \frac{\exp {\Bigl(\frac{- E(\mathbf{r})}{kT}\Bigr)}}{\sum_{i} \exp {\Bigl(\frac{- E(\mathbf{r_{i}})}{kT}\Bigr)} }$$

where $E(\mathbf{r})$ is the energy of a configuration $\mathbf{r}$.  In MC simulations, the statistical weight of moving from a configuration, $\mathbf{r_1}$, to a new configuration, $\mathbf{r_2}$, is:

$$W(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \frac{W(\mathbf{r_1})P(\mathbf{r}_1 \rightarrow \mathbf{r}_2)}{N}$$

where $W(\mathbf{r_1})$ is the weight associated with $\mathbf{r}$, $P(\mathbf{r}_1 \rightarrow \mathbf{r}_2)$ is the probability of moving from configuration $\mathbf{r}_1$ to $\mathbf{r}_2$ and *N* is the number of possible configurations. The corresponding weight of going from $\mathbf{r}_2$ back to $\mathbf{r}_1$ is:

$$W(\mathbf{r}_2 \rightarrow \mathbf{r}_1) = \frac{W(\mathbf{r_2})P(\mathbf{r}_2 \rightarrow \mathbf{r}_1)}{N}$$   

<img src="images/Tut_2_images/weights.png" height='400' width='600'/>

<div style="text-align: center">**Figure 1:** The associated statistical weights of moving between two configurations, A and B.</div>

There are more sophisticated ways of sampling configurational space, such as the Metropolis Algorithm, which is one of the most widely used sampling schemes in MC simulations (including this one).  First, it randomly selects a particle in the system and proposes a move to another configuration.  It then calculates the new energy of the configuration and compares it with the energy of the previous configuration before the move was proposed.  It then applies the following condition:

$$P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \min(1, \exp \ \Bigl(- \frac{E(\mathbf{r}_2) - E(\mathbf{r}_1)}{kT}\Bigr) \ )$$

where $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2)$ is the probability of accepting the move from the initial configuration, $\mathbf{r}_1$, with an energy, $E(\mathbf{r}_1)$, to the new configuration, $\mathbf{r}_2$, with an energy, $E(\mathbf{r}_2)$.  The function min() means that the smallest value in the brackets is chosen.  If the energy of the new configuration is less than that of the original, *i.e.* $E(\mathbf{r}_2) < E(\mathbf{r}_1)$, then $E(\mathbf{r}_2)-E(\mathbf{r}_1) < 0$ and so $\exp \ \Bigl(- \frac{E(\mathbf{r}_2) - E(\mathbf{r}_1)}{kT}\Bigr) \  > 1$ and so the move is accepted with $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = 1$.  If the new energy is greater than the energy of the original configuration, *i.e.* $E(\mathbf{r}_2) > E(\mathbf{r}_1)$, then $E(\mathbf{r}_2)-E(\mathbf{r}_1) > 0$ and so $\exp \ \Bigl(- \frac{E(\mathbf{r}_2) - E(\mathbf{r}_1)}{kT}\Bigr) \  > 1$ and the move is accepted with probability $P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = \exp \ \Bigl(- \frac{E(\mathbf{r}_2) - E(\mathbf{r}_1)}{kT}\Bigr) \ < 1$.  

<img src="images/Tut_2_images/Metropolis_algorithm.png" />

<div style="text-align: center">**Figure 2:** Visual representation of the function of the Metropolis algorithm.  Once one move outcome is complete, the algorithm repeats on the final configuration. </div>

Even if the proposed move leads to a higher-energy configuration, there is still a non-zero probability of it being accepted! Why should this be the case?

In [10]:
a = input()

What happens to the total number of accepted moves in a given simulation as we change the temperature? How might this affect the final outcome of your simulation?

In [11]:
b = input()

This defines the concept of detailed balance:

$$W(\mathbf{r}_1 \rightarrow \mathbf{r}_2)P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2) = W(\mathbf{r}_2 \rightarrow \mathbf{r}_1)P_{\mathrm{acc}}(\mathbf{r}_2 \rightarrow \mathbf{r}_1)$$

We can now obtain the required Boltzmann distribution from this condition by rearrangement:

$$\frac{W(\mathbf{r}_2 \rightarrow \mathbf{r}_1)}{W(\mathbf{r}_1 \rightarrow \mathbf{r}_2)} = \frac{P_{\mathrm{acc}}(\mathbf{r}_1 \rightarrow \mathbf{r}_2)}{P_{\mathrm{acc}}(\mathbf{r}_2 \rightarrow \mathbf{r}_1)} = exp \ {\Bigl(\frac{E_2 -E_1}{kT}\Bigr)}$$ 

This tells us that so long as we satisfy detailed balance, our system will be sampled according to the Boltzmann distribution and obey the rules of thermodynamics.  Though it is important to note that the condition of detailed balance is *sufficient* but *not necessary* to ensure that are system accurately reflects thermodynamics, *i.e.* there are other simpler conditions one could employ that would ensure that our simulation obeys thermodynamics.  For instance, one could ensure that *balance* is achieved from the system which simply states that moving from one state to another state is the same for any initial and final state pairing, *i.e.*:
   
$$\frac{\mathrm{d}W(\mathbf{r}_1)}{\mathrm{d}t} = 0$$

However, detailed balance also ensures equilibrium between all states such that the trajectory from one configuration to another via several steps has the same probability as the reverse trajectory.  This ensures the reliability of the sampling method used without requiring additional corrections in the calculations.

<img src="images/Tut_2_images/detailed_balance2.png" height='700' width='700'/>

<div style="text-align: center">**Figure 3:** A visualisation of the difference between the condition of balance (left) and detailed balance (right) for a set of different configurations, A-H, in the configurational space of a system.  </div>

Having discussed the concepts behind MC simulation methods, it is time to demonstrate how to apply them to a physical system.  This tutorial will be centred on a MC simulation of the magnetic properties of solid materials.

## Ising Model of Magnetism

An application where MC is more effective than deterministic methods is simulating the magnetic behaviour of solid state materials.  

Our simulation will be based on a 2D Ising model, which describes the macroscopic magnetic behaviour of a solid material as a result of the relative orientation of electron spins within the crystal lattice of a material.  As you may recall, each electron has an intrinsic 'spin'.  In simple terms, the spin of an electron can be thought of as a magnetic moment, with two possible orientations: 'up' and 'down'.  This idea helps define two classes of magnetic materials: diamagnetic and paramagnetic.  

Diamagnetic materials are made up of atoms/molecules without unpaired electrons, do not interact with external magnetic fields, making them non-magnetic.  Paramagnetic materials contain unpaired electrons, exhibiting a net magnetic moment that can interact with external magnetic fields and give the material its magnetic properties.  Figure 1 below shows an example of a paramagnetic material as a 2D lattice of colour-coded spins.

<img src="images/Tut_2_images/paramagnet_config.png" />

<div style="text-align: center">**Figure 4:** A 2D schematic of a paramagnetic material under an external magnetic field.  Yellow indicates the spins that are aligned with the field and purple are spins that are anti-aligned.  </div>

There is another type of magnetism observed known as ferromagnetism, where instead of a uniform alignment of spins as in paramagnetic materials, 'domains' of aligned spins form, bound by domains of oppositely aligned spins (see Figure 2).  Ferromagnetic materials can show unique properties, such as being able to generate their own magnetic field (magnetisation) in the absence of an external magnetic field.  These form the common magnets seen in real-world applications.

<img src="images/Tut_2_images/ferromagnet_cand2.png" />

<div style="text-align: center">**Figure 5:** A 2D schematic of a ferromagnetic material at $T < T_{c}$.  Yellow and purple represent the two different spin orientations, 'up' and 'down', respectively.  </div>

The main factor influencing whether a given atom's spin is aligned with its neighbours in a crystal, and hence what type of magnetism the material displays, is its exchange energy, *E*, which in the Ising model is given by:

$$E = -J \sum_{<i,j>} s_{i}s_{j}$$

where *J* is the coupling constant between adjacent atoms in a given material and $s_{i/j}$ is the spin of the particle in position i/j in the lattice, respectively.  The <...> here mean the sum goes over the nearest neighbours of the atom in position (i,j), *i.e.* over the atoms at positions  (i-1, j), (i+1, j), (i, j-1) and (i, j+1) only.  The sign of *J* determines whether spin alignment (ferromagnetism) or anti-alignment (antiferromagnetism) is favourable.

The exchange energy can be thought of as an activation barrier for an atom to change its spin depending on the spins of its neighbours.  This means that, like with any physical system with an energy barrier, spontaneous thermal fluctuations can overcome the barrier and cause some atoms/domains to flip their spin, with the likelihood of flipping a spin increasing as temperature increases.  Therefore, ferromagnetic materials only show domains at temperatures under a specific critical, or Curie, temperature, $T_{c}$.  

Above this point, ferromagnetic materials lose their ability to retain magnetisation because the thermal fluctuations are much larger than the energy required to switch a domain's alignment with respect to other domains.  This results in a loss of the domain structure, and hence loss of magnetisation without an external field.  It is for this reason that paramagnetism can be thought of as high-temperature ferromagnetism.

For more information on the Ising model, consult either [2] or [3].

The Metropolis algorithm is employed in these simulations, describe what constitutes a 'move' in the context of this system.

In [ ]:
c = input()

Write an expression for the energy difference between the initial and final configurations, $E(\mathbf{r}_2) - E(\mathbf{r}_1)$, for the 2D Ising model.

In [ ]:
d = input()

### Exercise 1)

The aim of this exercise is to familiarise yourself with running MC calculations on a simple 2D Ising model of a ferromagnetic material. The material is represented by a 64x64 2D lattice of points, each representing an atom with its own net spin.  In this exercise, all atoms are spin-aligned.  We will be running a MC simulation to look at how the overall spin alignment (magnetisation) and energy of the system evolves with both time and temperature.

First, we shall setup our intial simulation at a given temperature:

In [2]:
data = dlmonte.DLMonteData("")

Now let's run our first Monte Carlo simulation of the day!

In [3]:
# Run the initial simulation.  Takes about a minute to complete

If you wish, you can look in your directory and see several new files have appeared.  The nature of these files will be explained in detail next session.  

Now that you have all the output data you could possibly need from this calculation, we shall proceed with extracting the time evolution of magnetisation and the distribution of the magnetisations over the course of the simulation. 

In [9]:
# output data extraction and analysis into plots of magnetisation vs time and histogram of magnetisation distributions

T = 2.36
plt.figure()
plt.subplot(1,2,1)
plt.xlabel("Number of steps")
plt.ylabel("Magnetisation")
plt.title("Time evolution of magnetisation at T = {}".format(T))
plt.axis()
plt.plot(M_seq.dat, 'b-')

plt.subplot(1,2,2)
plt.xlabel("M")
plt.ylabel("P(M)")
plt.title("Distribution of magnetisations at T = {}".format(T))
plt.hist(M_seq.dat, bins=auto, normed=True, weights=None)

NameError: name 'M_seq' is not defined

You will find several new files in your directory, but we will have used only on M_seq.dat and M_hist.dat in this exercise (we will get to the others later).

We shall now proceed to run the calculation at higher temperatures to obtain the temperature-dependence of the magnetisation.  Repeat the simulation and analysis sections that you have done for this initial temperature with the other temperatures in the main directory.

Compare the evolution of magnetisation as the temperature changes and rationalise any observed trends using your knowledge of ferromagnetism.  Do the results correspond to the Ising model?

In [1]:
e = input()

Compare the shapes of your magnetisation histograms as the temperature changes.  What does this indicate is happening to your system as temperature changes? Does this behaviour support the Ising model and your magnetisation evolution data?

In [2]:
f = input()

Once you have done that, plot magnetisation vs temperature for the system.  Comment on the shape of your graph and estimate the critical temperature, $T_{c}$, from it. 

In [3]:
g = input()

In [5]:
# collate all magnetisation-temperature data and plot it

For any square 2D Ising model where coupling along rows and along columns are equal, $T_{c}$ is given by:

$$T_{c} = \frac{2}{\ln(1+\sqrt{2})} \approx 2.269$$

Does your estimation of $T_{c}$ agree with that predicted by the above equation? Account for any observed discrepancies.  How could you improve the accuracy of your estimated value for $T_{c}$?

In [4]:
h = input()

### Extension (optional):

You have seen what happens as the system is heated, but you can also look at the magnetisation upon cooling the system from a state above the critical temperature to a state below the critical temperature. 

Go back to the beginning of Exercise 1 and now choose the inputs in ------- and plot the time-evolution of magnetisation.

How does this compare with the time evolution at $T>T_{c}$? Does this agree with the Ising model? If not, what do you think might be the problem with our simulation?

In [5]:
i = input()

### Exercise 2)

This exercise will demonstrate the stochastic nature of MC simulations as well as how the Metropolis algorithm produces reliable and accurate results for this simple 2D Ising model.

We have seen what happens when we start the simulations from a fixed starting configuration (all spins aligned), but what will happen when we start from a random configuration?

Go back the the beginning of Exercise 1 and repeat it for each temperature in the 'ranseed' folder, plotting the magnetisation vs. temperature once you have run all the simulations.  

How do the results from this exercise compare with those of Exercise 1? What effect does the initial configuration have on the outcome of the simulation?

In [6]:
j = input()

### Extension (optional):

For one of the ranseed calculations, let us find out what the initial configuration was and use that as our fixed starting configuration and see how the results of both calculations compare.  Using your current ranseed calculation, we will extract the initial configuration and set it as the starting configuration in the CONTROL file:

In [1]:
# pull out seeds, copy input files into a new directory, change ranseed to seeds in output

Now let's run this calculation:

In [2]:
# Run calculations here

We can compare the magnetisation data between the ranseed and this new simulation:

In [3]:
# plot both magentisations on the same graph

What do you notice about the magnetisation evolution in the two calculations? Does this confirm that the stochastic nature of Monte Carlo methods can produce reliable results?

In [7]:
k = input()

## Conclusions:

Now that you have reached the end of this tutorial, you will hopefully have a better understanding of the Monte Carlo method and the motivation for its use. You have simulated the magnetic properties of a 2D material based on the Ising model and obtained:

- the temperature-dependence of magnetisation
- the evolution of magnetisation with time
- validation of the stochastic nature of Monte Carlo methods

In the next tutorial, you will be introduced to a general Monte Carlo program called DLMONTE and use it to model the thermal properties of a Lennard-Jones material.

## Extensions (optional):

### 1. Antiferromagnetism:

So far, you have looked at how the magnetic behaviour of a ferromagnetic system changes over time and temperature, but there is another possible type of magnetism called antiferromagnetism, where the sign of the coupling constant, *J*, changes sign.  This means that it is now favourable for the spin of one atom to be opposed to the spin of its neighbours, resulting in a preferred 'checkerboard' pattern of magnetisation on the 2D lattice (see Figure 3).  You can investigate the magnetic behaviour in this case using the 2D Ising model.

<img src="images/Tut_2_images/antiferromagnet.png" />

<div style="text-align: center">**Figure 6:** The most stable magnetic configuration of an antiferromagnetic material at $T < T_{c}$. </div>

Repeat Exercise 1 but this time using the inputs in the 'antiferromagnet' folder, plotting the temperature dependence of the magnetisation once you have run the simulation at each temperature.

Compare your results of the antiferromagnet with the ferromagnet.  Rationalise any observed differences in terms of exchange energy and alignment of spins.

In [8]:
l = input()

## References:

[1] S. Mordechai (Editor), *Applications of Monte Carlo Method in Science and Engineering* [Online]. Available: https://www.intechopen.com/books/applications-of-monte-carlo-method-in-science-and-engineering 

[2] J. V. Selinger, "Ising Model for Ferromagnetism" in *Introduction to the Theory of Soft Matter: From Ideal Gases to Liquid Crystals*.  Cham: Springer International Publishing, 2016, pp. 7-24.

[3] N. J. Giordano, *Computational Physics*.  Upper Saddle River, N.J.: Prentice Hall, 1997. 